### Input

* data/processed/etl_1-3_hw_identifying_tb.csv
* data/processed/etl_2_twoyo_under_tack.csv

### Output
* data/processed/analysis_deaths_by_state_racetrack_2023.csv
* data/processed/analysis_undertack_counts.csv
* data/processed/analysis_state_counts_2023.csv

In [1]:
import pandas as pd
import numpy as np

processed = "data//processed//"
manual = "data//manual//"


deaths_w_breed = pd.read_csv(processed+"etl_1-3_hw_identifying_tb.csv")
tb_2023 = deaths_w_breed[deaths_w_breed.TB.isin(['T', 'T and Q'])]

print("\nDEATHS BY BREED")
print(deaths_w_breed.TB.value_counts())
print("\nTHOROUGHBRED & 'T & Q' HORSES")
print(tb_2023['name'].count())

tb_2023.to_csv(processed + "analysis_tb_2023_racing_training.csv", index = None)

by_state = pd.pivot_table(tb_2023, index='State', columns = 'race_train_stall', values = 'name', aggfunc='count').reset_index()
by_state['T'] = by_state['T'].replace(np.nan, 0)
by_state.columns.name = None
by_state.columns = ['State', 'Racing Deaths', 'Training Deaths']
by_state['Total Deaths'] = by_state['Racing Deaths'] + by_state['Training Deaths']
by_state.to_csv(processed + "analysis_state_counts_2023.csv", index=None)

by_state_and_track = tb_2023.groupby(['State', 'track'])['name'].count().reset_index()
by_state_and_track.columns = ['State', 'Track', '2023 # of Thoroughbreds Dead from Training or Racing (so far)']
by_state_and_track.to_csv(processed + "analysis_deaths_by_state_racetrack_2023.csv",index=None)

pd.DataFrame(["SOURCE: Scripps News Analysis of Horseracing Wrongs, Equibase, and Pedigree Query data"]).to_csv(processed + "analysis_deaths_by_state_racetrack_2023.csv", mode= 'a', index=None)


DEATHS BY BREED
TB
T          295
Q           35
H           12
T and Q      3
unclear      2
Name: count, dtype: int64

THOROUGHBRED & 'T & Q' HORSES
298


In [2]:
by_state

,State,Racing Deaths,Training Deaths,Total Deaths
0,Arizona,11.0,6.0,17.0
1,Arkansas,9.0,6.0,15.0
2,California,13.0,22.0,35.0
3,Delaware,5.0,0.0,5.0
4,Florida,7.0,0.0,7.0
5,Illinois,6.0,5.0,11.0
6,Indiana,3.0,1.0,4.0
7,Kentucky,24.0,11.0,35.0
8,Louisiana,10.0,0.0,10.0
9,Maryland,7.0,8.0,15.0


In [3]:
print("\n2023 THOROUGHBRED RACING OR\nTRAINING RELATED DEATHS")
print(tb_2023.State.value_counts(dropna=False))


2023 THOROUGHBRED RACING OR
TRAINING RELATED DEATHS
State
New York         47
Kentucky         35
California       35
West Virginia    28
Pennsylvania     21
Ohio             20
Arizona          17
Arkansas         15
Maryland         15
Illinois         11
Texas            10
Louisiana        10
Florida           7
New Mexico        6
New Jersey        5
Delaware          5
Indiana           4
Virginia          2
Oregon            2
Wyoming           1
North Dakota      1
Washington        1
Name: count, dtype: int64


## Under tacking

Mostly trying to figure out whether we can use the number of horses sold at the Bloodhorse auctions as a measure of the number of horses that breezed. (basically trying to figure out if the number of horses that breezed+were withdrawn is larger than the number of horses that were not breezed + sold/RNAed)

In [4]:
FULL_DATA_TYPES = ["FT","OBS","API"]
undertack = pd.read_csv(processed+"etl_2_twoyo_under_tack.csv")
undertack["full_data"] = undertack["type"].isin(FULL_DATA_TYPES)
full_data_ut = undertack[undertack["full_data"]]

print("Far fewer horses are sold or rna at undertack shows without being breezed, than are breezed and withdrawn.")
print(full_data_ut.pivot_table(index="breezed",columns="sale_status",values="file",aggfunc="count"))

Far fewer horses are sold or rna at undertack shows without being breezed, than are breezed and withdrawn.
sale_status   na     rna     sold  withdrawn
breezed                                     
False        1.0    49.0    168.0     5554.0
True         NaN  3025.0  12336.0     1419.0


In [5]:
# Summarize by file
def get_counts(df):
    return pd.Series([len(df), (df["sale_status"]=="sold").sum(), (df["breezed"]=="True").sum(), ((df["sale_status"]=="sold")&(df["breezed"]=="False")).sum(), ((df["sale_status"]!="sold")&(df["breezed"]=="True")).sum()], index=["total","sold","breezed","sold_not_breezed","breezed_not_sold"])


ut_counts = undertack.groupby(["file","full_data"]).apply(get_counts).reset_index()
ut_counts.to_csv(processed+"analysis_undertack_counts.csv",index=None)

print("For the undertack shows where we have full data on horses sold and horses breezed, the number of horses sold and not breezed is always lower than the number of horses breezed and not sold.")
full_data_counts = ut_counts[ut_counts["full_data"]]
full_data_counts["sold_underest_breezed"] = full_data_counts["breezed_not_sold"] > full_data_counts["sold_not_breezed"]
full_data_counts

For the undertack shows where we have full data on horses sold and horses breezed, the number of horses sold and not breezed is always lower than the number of horses breezed and not sold.


/var/folders/n1/qf74t1dj555cgs6f_mhz9nm0m6xrwl/T/ipykernel_90323/3963028621.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data_counts["sold_underest_breezed"] = full_data_counts["breezed_not_sold"] > full_data_counts["sold_not_breezed"]


,file,full_data,total,sold,breezed,sold_not_breezed,breezed_not_sold,sold_underest_breezed
2,Apr19_Excel.xls,True,1219,674,1001,0,327,True
3,Apr20_Excel.xls,True,1312,634,777,1,144,True
4,Apr21_Excel.xls,True,1214,724,864,0,140,True
5,Apr22_Excel.xlsx,True,1230,705,900,0,195,True
6,Apr23_Excel.xls,True,1222,699,876,0,177,True
7,April18_Excel.xls,True,1220,698,959,0,261,True
12,Jul20_Excel.xls,True,1110,523,639,13,129,True
13,Jun19_Excel.xls,True,1058,624,749,19,144,True
14,Jun21_Excel.xls,True,926,575,733,8,166,True
15,Jun22_Excel.xls,True,1130,671,888,18,235,True


In [6]:
# Summarize by file
def get_counts(df):
    return pd.Series([len(df), (df["sale_status"].isin(["sold","rna"])).sum(), (df["breezed"]=="True").sum(), ((df["sale_status"].isin(["sold","rna"]))&(df["breezed"]=="False")).sum(), ((df["sale_status"]=="withdrawn")&(df["breezed"]=="True")).sum()], index=["total","sold+rna","breezed","sold+rna_not_breezed","breezed_withdrawn"])


ut_counts = undertack.groupby(["file","full_data"]).apply(get_counts).reset_index()
#ut_counts.to_csv(processed+"analysis_1_undertack_counts.csv",index=None)

print("Same thing but sold+rna")
full_data_counts = ut_counts[ut_counts["full_data"]]
full_data_counts["sold_underest_breezed"] = full_data_counts["breezed_withdrawn"] > full_data_counts["sold+rna_not_breezed"]
full_data_counts.sort_values("sold_underest_breezed")

Same thing but sold+rna


/var/folders/n1/qf74t1dj555cgs6f_mhz9nm0m6xrwl/T/ipykernel_90323/2642894085.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_data_counts["sold_underest_breezed"] = full_data_counts["breezed_withdrawn"] > full_data_counts["sold+rna_not_breezed"]


,file,full_data,total,sold+rna,breezed,sold+rna_not_breezed,breezed_withdrawn,sold_underest_breezed
40,https://tcapiodata.azurewebsites.net/odata/Sal...,True,193,167,148,25,6,False
3,Apr20_Excel.xls,True,1312,778,777,1,0,False
38,https://tcapiodata.azurewebsites.net/odata/Sal...,True,121,104,99,8,3,False
39,https://tcapiodata.azurewebsites.net/odata/Sal...,True,144,119,111,14,6,False
21,Mar20_Excel.xls,True,681,482,481,1,0,False
12,Jul20_Excel.xls,True,1110,654,639,15,0,False
13,Jun19_Excel.xls,True,1058,768,749,19,0,False
32,Santa Anita Two-Year-Olds in Training (2019).csv,True,168,108,93,20,5,False
17,June18_Excel.xls,True,934,673,664,9,0,False
29,Midlantic Two-Year-Olds in Training (2020).csv,True,559,375,401,9,35,True


In [7]:
horses_breezed = undertack[undertack["breezed"]=="True"]["file"].count()
horses_sold_bh_data = undertack[(undertack["type"]=="BH")&(undertack["sale_status"]=="sold")]["file"].count()
print("Horses breezed:",horses_breezed)
print("Horses sold in BH auctions (no breeze data):",horses_sold_bh_data)
print("both",horses_breezed+horses_sold_bh_data)

Horses breezed: 16990
Horses sold in BH auctions (no breeze data): 223
both 17213
